## Lecture CSV

In [10]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

In [11]:
# Lecture du CSV
df_sd = pd.read_csv("../datas/SpeedDatingData.csv", encoding = "ISO-8859-1")
df_sd.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


---

## Analyse - Axe 2

### A quels attributs les participants accordent-ils le plus d’importance dans la décision de revoir un partenaire ?

In [12]:
# Sélection des colonnes pertinentes pour l'analyse : décision de revoir le partenaire + évaluations (de 1 à 10) des partenaires sur la base de 6 attributs
columns_to_keep = ["dec", "attr","sinc", "intel", "fun", "amb", "shar"]

# Vérifier les valeurs manquantes
missing_values = df_sd[columns_to_keep].isnull().sum()
print("Valeurs manquantes : ")
print(missing_values)

print("")

print("Pourcentage de valeurs manquantes : ")
print(100 * missing_values / df_sd.shape[0])

Valeurs manquantes : 
dec         0
attr      202
sinc      277
intel     296
fun       350
amb       712
shar     1067
dtype: int64

Pourcentage de valeurs manquantes : 
dec       0.000000
attr      2.411077
sinc      3.306278
intel     3.533063
fun       4.177608
amb       8.498448
shar     12.735736
dtype: float64


Le pourcentage de valeurs manquantes est important.

Je vais imputer les valeurs manquantes par la médiane car c'est la valeur centre la plus représentative et la moins impactée par les valeurs extrêmes.

In [13]:
df_sd_clean = df_sd[columns_to_keep]
df_sd_clean = df_sd_clean.apply(lambda col: col.fillna(col.median()))

# Vérifier que les valeurs manquantes ont été remplacées
df_sd_clean.isnull().sum()

dec      0
attr     0
sinc     0
intel    0
fun      0
amb      0
shar     0
dtype: int64

Le dataset est propre

In [14]:
# On remplace la décision de revoir par "Oui" ou "Non", pour la visualisation
df_sd_clean["dec"] = df_sd_clean["dec"].apply(lambda x: "Oui" if x == 1 else "Non")
display(df_sd_clean)

,dec,attr,sinc,intel,fun,amb,shar
0,Oui,6.0,9.0,7.0,7.0,6.0,5.0
1,Oui,7.0,8.0,7.0,8.0,5.0,6.0
2,Oui,5.0,8.0,9.0,8.0,5.0,7.0
3,Oui,7.0,6.0,8.0,7.0,6.0,8.0
4,Oui,5.0,6.0,7.0,7.0,6.0,6.0
...,...,...,...,...,...,...,...
8373,Non,3.0,5.0,5.0,5.0,7.0,6.0
8374,Non,4.0,6.0,8.0,4.0,4.0,6.0
8375,Non,4.0,7.0,8.0,8.0,8.0,6.0
8376,Non,4.0,6.0,5.0,4.0,7.0,5.0


Nous allons analyser la moyenne de la notation de chaque attribut selon la décision de revoir

In [15]:
# Moyenne de la notation de chaque attribut selon la décision de revoir
df_mean = df_sd_clean.groupby("dec")[["attr", "sinc", "intel", "fun", "amb", "shar"]].mean().reset_index()
display(df_mean)

,dec,attr,sinc,intel,fun,amb,shar
0,Non,5.392181,6.865021,7.072119,5.773148,6.541770,4.908025
1,Oui,7.281239,7.589824,7.747157,7.327032,7.148238,6.416572


In [16]:
# on retravaille le dataframe pour avoir, pour chaque décision, l'attribut, la moyenne des notes pour cet attribut
melted_df = df_mean.melt(id_vars=["dec"], value_vars=["attr", "sinc", "intel", "fun", "amb", "shar"])
melted_df["variable"] = melted_df["variable"].apply(lambda x: "Attractivité" if x == "attr" 
                                                    else "Sincérité" if x == "sinc"
                                                    else "Intelligence" if x == "intel"
                                                    else "Humour" if x == "fun"
                                                    else "Ambition" if x == "amb"
                                                    else "Intérêts communs" if x == "shar" else "Aucun")
display(melted_df)

,dec,variable,value
0,Non,Attractivité,5.392181
1,Oui,Attractivité,7.281239
2,Non,Sincérité,6.865021
3,Oui,Sincérité,7.589824
4,Non,Intelligence,7.072119
5,Oui,Intelligence,7.747157
6,Non,Humour,5.773148
7,Oui,Humour,7.327032
8,Non,Ambition,6.541770
9,Oui,Ambition,7.148238


In [17]:
#Visualisation des moyennes des notes des attributs selon la décision prise
fig = px.bar(melted_df, x="variable", y="value", color="dec", barmode="group", 
             title="Moyenne des attributs selon la décision de revoir", color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(title_x=0.5, xaxis_title="Attributs",  yaxis_title="Valeur Moyenne", legend_title="Décision de revoir")


L'`attractivité`, l’`humour` et les `intérêts communs` ont des écarts plus marqués entre les participants qui sont souhaités être revus et ceux qui ne le sont pas.

Cela pourrait indiquer que ces critères sont particulièrement importants dans la prise de décision contrairement à la sincérité, l'intelligence et l'ambition.

Nous allons donc analyser plus précisémment la distribution des notes pour les 3 attributs : attractivité, humour et intérêts communs.

In [18]:
#Distribution des attributs Attractivité, Humour et Intérêts communs selon la décision

attributes = {"attr" : "Attractivité", "fun" : "Humour", "shar" : "Intérêts communs"}

# Création des subplots
fig = make_subplots(rows = 1, cols = 3, subplot_titles=[attributes[attr] for attr in attributes])

mask_oui = (df_sd_clean["dec"] == "Oui")
mask_non = (df_sd_clean["dec"] == "Non")

# Ajout des boxplots dans les subplots
for i, attr in enumerate(attributes):
    # Décision Oui
    fig.add_trace(
        go.Box(y=df_sd_clean[mask_oui][attr], name="Oui", marker_color=px.colors.qualitative.Pastel[0]),
        row=1, col=i+1
    )
    # Décision Non
    fig.add_trace(
        go.Box(y=df_sd_clean[mask_non][attr], name="Non", marker_color=px.colors.qualitative.Pastel[1]),
        row=1, col=i+1
    )

fig.update_layout(title="Distribution des notes des attributs selon la décision", title_x=0.5, height=500, width=1200, showlegend=False)
fig.show()

Analyse des boxplots


1. Attractivité
- La boîte est plus resserrée pour les "Oui" → jugements plus homogènes.
- Les "Oui" ont peu de notes faibles.
- Les "Non" ont une large variation (0 à 10), donc plus de diversité d'opinion.

Conclusion : Une attractivité perçue plus élevée est clairement associée à la décision positive. C’est un critère fort de sélection.

2. Humour
- Les boîtes "Oui" et "Non" sont toutes les deux resserrées → jugements plus homogènes.
- Les "Non" sont un peu plus étalés.

Conclusion : L’humour joue un rôle important dans la décision. C'est également un critère qui est jugé de manière homogène par les participants.

3. Intérêts communs
- Les "Oui" ont une large variation (0 à 10), donc plus de diversité d'opinion.
- La boîte est plus resserrée pour les "Non" → jugements plus homogènes.

Conclusion : Les intérêts partagés semblent un critère important. Il y a cependant moins d'homogénéité dans la notation.

Analyse croisée

Bien que les médianes des notes d’attractivité, d’humour et d’intérêts communs soient systématiquement plus élevées pour les décisions "Oui", les distributions entre "Oui" et "Non" présentent un certain chevauchement.

Cela indique que ces attributs, bien qu’influents, ne suffisent pas à eux seuls pour garantir une décision positive.